In [ ]:
from datetime import datetime

from mcs_shared import (
    load_als_depth, ALL_FLIGHT_DATES, model_run, get_raster_pixel_value,
    mcs_snotel_depth, x_y_snotel
)
from common import *

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
RESOLUTION = 10

# About
Show measured ALS flight depth compared to the SNOTEL station depth

## SNOTEL 

In [ ]:
snotel_depth, mcs_snotel = mcs_snotel_depth(datetime(2020, 10, 1), datetime(2025, 7, 31))
utm_x, utm_y = x_y_snotel(mcs_snotel)

## Flights

In [ ]:
mcs_dates = [pd.to_datetime(date) for date in ALL_FLIGHT_DATES]

In [ ]:
def get_flights(dates):
    als_depth = []
    means = []

    for flight in dates:
        mcs_als = load_als_depth(flight, RESOLUTION)
        mean = float(np.nanmean(mcs_als))

        als_flight = f"{model_run(RESOLUTION)}/{flight}_MCS-snowdepth_{RESOLUTION}m.tif"
        
        means.append(mean)
        als_depth.append(
            get_raster_pixel_value(als_flight, utm_x, utm_y)       
        )

    return als_depth, means

In [ ]:
mcs_als, means = get_flights(ALL_FLIGHT_DATES)

In [ ]:
flight_values = [[date, float(flight)] for date, flight in zip(mcs_dates, mcs_als)]

## Plots

### Matplotlib

In [ ]:
als_values = [float(flight) for flight in mcs_als]

In [ ]:
fig, ax = plt.subplots(dpi=300, figsize=(10,4))

snotel_depth['SNOWDEPTH'].plot(
    label='Station', 
    ax=ax,
    color='seagreen', lw=1
)
ax.scatter(
    mcs_dates, als_values,
    s=20,
    color='black',
    label='ALS',
    zorder=10
)
ax.scatter(
    [mcs_dates[4], mcs_dates[5], mcs_dates[6], mcs_dates[10], mcs_dates[12], mcs_dates[13]], 
    [als_values[4], als_values[5], als_values[6], als_values[10], als_values[12], als_values[13]],
    s=40,
    color='orange',
    zorder=10
)
ax.set_title('')

ax.set_ylabel('Snow Depth (m)')
ax.set_ylim(-0.1, 3)

ax.set_xlabel('')
ax.set_xlim(pd.to_datetime('2020-10-01'), pd.to_datetime('2025-07-31'))

ax.legend(ncols=3);

### Holoviews

In [ ]:
use_hvplot()

In [ ]:
hv.output(fig='auto', dpi=300)

HV_PLOT_OPTS['ylabel'] = 'Snow Depth (m)'
HV_PLOT_OPTS['xlabel'] = f'Water Year'

snotel_depth['SNOWDEPTH'].plot(label='Station', color='seagreen', line_width=3, ylim=(-0.05, 3.), **HV_PLOT_OPTS) * \
hv.Points(flight_values, label='ALS').opts(color='black', size=10, tools=['hover'])